In [18]:
#!pip install -q cassio datasets langchain openai tiktoken
! python -m pip install cassio
#! python -m pip install PyPDF2

In [3]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:PKsGdrFjhFwwpFDAoUPEGrDZ:d2ae0fe496f1fa598efb77b39f402ca310f4031ae7c6764bc155bb3b9a5b45f1"
ASTRA_DB_ID="58ed3148-75b3-49a9-a946-93bbaf87b3c2"

In [20]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_huggingface import HuggingFaceEmbeddings,HuggingFaceEndpoint
import cassio

In [ ]:
repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id,huggingfacehub_api_token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [11]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [19]:
hf

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

In [21]:
from PyPDF2 import PdfReader
# provide the path of  pdf file/files.
pdfreader = PdfReader(r"F:\Langchain\langchainprojects\research_papers\apjspeech.pdf")

In [22]:
from typing_extensions import Concatenate
raw_text=""

for i, page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [23]:
raw_text

'A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowing important messages: \n \n1. Accelerate development : Aspiration of th

In [24]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [25]:
astro_vector_store=Cassandra(
    embedding=hf,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,

)

In [26]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [27]:
texts[:50]

['A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at',
 'home makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline

In [28]:
astro_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))
astro_vectore_index=VectorStoreIndexWrapper(vectorstore=astro_vector_store)

Inserted 31 headlines.


In [29]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astro_vectore_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astro_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


QUESTION: "who is APJ"
ANSWER: "APJ refers to APJ Abdul Kalam, the former President of India, who is the one having the experiences mentioned in the text."

FIRST DOCUMENTS BY RELEVANCE:
    [0.5784] "benefits of technology to the people has enabled us to work with Afric a to bring so ..."
    [0.5750] "academics and rasikas. Now he wants to give, give and give his art to inspire others ..."
    [0.5730] "On the evening of February 24, 2007, at Coimbatore, I had a very beautif ul experien ..."
    [0.5722] "that area. When I went there, I found that the school building had been severely  da ..."
